## osumapper: create osu! map using Tensorflow and Colab

### -- For osu!mania game mode --

For mappers who don't know how this colaboratory thing works:
- Press Ctrl+Enter in code blocks to run them one by one
- It will ask you to upload .osu file and audio.mp3 after the third block of code
- .osu file needs to have correct timing (you can use [statementreply](https://osu.ppy.sh/users/126198)'s TimingAnlyz tool)
- After uploading them, wait for a few minutes until download pops

Github: https://github.com/kotritrona/osumapper

### Step 1: Installation

First of all, check the Notebook Settings under Edit tab.<br>
Activate GPU to make the training faster.

Then, clone the git repository and install dependencies.

In [ ]:
%cd /content/
!git clone https://github.com/kotritrona/osumapper.git
%cd osumapper/v7.0
!apt install -y ffmpeg
!apt install -y nodejs
!cp requirements_colab.txt requirements.txt
!cp package_colab.json package.json
!pip install -r requirements.txt
!npm install

### Step 2: Choose a pre-trained model
Set the select_model variable to one of:

- "default": default model (choose only after training it)
- "lowkey": model trained with 4-key and 5-key maps (☆2.5-5.5)
- "highkey": model trained with 6-key to 9-key maps (☆2.5-5.5)

In [ ]:
from mania_setup_colab import *

select_model = "highkey"

model_params = load_pretrained_model(select_model);

### Step 3: Upload map and music file<br>
Map file = .osu file with correct timing (**Important:** Set to mania mode and the wished key count!)<br>
Music file = the mp3 file in the osu folder


In [ ]:
from google.colab import files
print("Please upload the map file:")
mapfile_upload = files.upload()
for fn in mapfile_upload.keys():
  uploaded_osu_name = fn
  print('Uploaded map file: "{name}" {length} bytes'.format(name=fn, length=len(mapfile_upload[fn])))
print("Please upload the music file:")
music_upload = files.upload()
for fn in music_upload.keys():
  print('Uploaded music file: "{name}" {length} bytes'.format(name=fn, length=len(music_upload[fn])))


### Step 4: Read the map and convert to python readable format



In [ ]:
from act_newmap_prep import *

step4_read_new_map(uploaded_osu_name);

### Step 5: Use model to calculate map rhythm

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"hold_favor" determines how the model favors holds against circles, ranges from -1 to 1.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"hold_max_ticks" determines the max amount of time a hold can hold off, ranges from 1 to +∞.<br>
"hold_min_return" determines the final granularity of the pattern dataset, ranges from 1 to +∞.<br>
"rotate_mode" determines how the patterns from the dataset gets rotated. modes (0,1,2,3,4)
- 0 = no rotation
- 1 = random
- 2 = mirror
- 3 = circulate
- 4 = circulate + mirror

In [ ]:
from mania_act_rhythm_calc import *

model = step5_load_model(model_file=model_params["rhythm_model"]);
npz = step5_load_npz();
params = model_params["rhythm_param"]
# Or set the parameters here...
# params = step5_set_params(note_density=0.6, hold_favor=0.2, divisor_favor=[0] * divisor, hold_max_ticks=8, hold_min_return=1, rotate_mode=4);

predictions = step5_predict_notes(model, npz, params);
notes_each_key = step5_build_pattern(predictions, params, pattern_dataset=model_params["pattern_dataset"]);

Do a little modding to the map.

Parameters:

- key_fix: remove continuous notes on single key modes (0,1,2,3) 0=inactive 1=remove late note 2=remove early note 3=divert

In [ ]:
modding_params = model_params["modding"]
# modding_params = {
#     "key_fix" : 3
# }

notes_each_key = mania_modding(notes_each_key, modding_params);
notes, key_count = merge_objects_each_key(notes_each_key)

Finally, save the data into an .osu file!

In [ ]:
from google.colab import files
from mania_act_final import *

saved_osu_name = step8_save_osu_mania_file(notes, key_count);

files.download(saved_osu_name)

In [ ]:
# clean up if you want to make another map!
# colab_clean_up(uploaded_osu_name)

That's it! Now you can try out the AI-created map in osu!.

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)

<img src="https://i.imgur.com/Ko2wogO.jpg" />